In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
d_bit=pd.read_csv("TweetsEtiquetados-año.csv")
d_bit

In [ ]:
d_bit['date'] = pd.to_datetime(d_bit['date']).dt.tz_localize(None)
tweets_grouped = d_bit.groupby([pd.Grouper(key='date', freq='M'), 'label']).size().unstack(fill_value=0)


In [ ]:
tweets_grouped

In [ ]:
#Se agrupa el dataset por clase y fecha: año y mes
#d_bit_agrupados=d_bit.groupby(['fechas','label'])['content'].count().reset_index()

In [ ]:
# se cogen solo las fechas desde 2013-01 a 2022-12
d_filtradas= tweets_grouped.drop(tweets_grouped.index[125:])
d_filtradas=d_filtradas.drop(d_filtradas.index[:6])

In [ ]:
#REllenamos el mes que falte con valor 0,0,0
# Definir rango de fechas esperado
start_date = '2013-01-01'
end_date = '2022-12-31'
expected_dates = pd.date_range(start=start_date, end=end_date, freq='M')

# Reindexar el dataset tweets_grouped con el rango de fechas esperado y rellenar los valores faltantes con 0
d_filtradas = d_filtradas.reindex(expected_dates, fill_value=0)


In [ ]:
d_filtradas=d_filtradas.rename_axis('date').reset_index()

In [ ]:
#Para contar el número de comentarios por año

#d_filtradas['tweets_total']=d_filtradas['N']+d_filtradas['NEU']+d_filtradas['P']
#d_filtradas['año']=d_filtradas['date'].dt.year
#suma_por_año = d_filtradas.groupby('año')['tweets_total'].sum()
#suma_por_año

In [ ]:
d_filtradas

## Añadir fichero bitcoin


In [ ]:
d_grafica_bit=pd.read_csv('BTC_ALL_graph_coinmarketcap.csv', sep=';')
d_grafica_bit

In [ ]:
d_grafica_bit['timestamp'] = pd.to_datetime(d_grafica_bit['timestamp']).dt.tz_localize(None)
d_grafica_bit= d_grafica_bit.sort_values('timestamp')

In [ ]:
d_grafica_bit.rename(columns={'timestamp': 'date'},inplace=True)

In [ ]:
d_grafica_bit

In [ ]:
#filtramos solo al rango de años con tweets
filtered_bitcoin_date= d_grafica_bit[(d_grafica_bit['date'] >= '2012-12-31') & (d_grafica_bit['date'] <= '2022-12-31')]

In [ ]:
bitcoin_mes = filtered_bitcoin_date.resample('M', on='date').mean()

In [ ]:
bitcoin_mes

In [ ]:
#Para sacar el precio medio anual
#d_grafica_bit['año'] = d_grafica_bit['date'].dt.year
#precio_anual = d_grafica_bit.groupby('año')['close'].mean()
#precio_anual

## Merge de datasets


In [ ]:
# Combinar los datos de tweets y Bitcoin en un solo DataFrame basado en la fecha
combined_data = pd.merge(d_filtradas, bitcoin_mes, on='date')

# Calcular la correlación entre las columnas de tweets y el precio del Bitcoin
correlation = combined_data[['P', 'N', 'NEU', 'close']].corr()

# Visualizar la matriz de correlación
plt.figure(figsize=(8, 6))
plt.imshow(correlation, cmap='coolwarm', interpolation='nearest')
plt.colorbar()
plt.title('Correlación entre tweets y precio del Bitcoin')
plt.xticks(range(correlation.shape[0]), correlation.columns, rotation=45)
plt.yticks(range(correlation.shape[1]), correlation.columns)
plt.show()


In [ ]:
combined_data

In [ ]:
combined_data=combined_data[['date','N','NEU','P','close']]

In [ ]:
combined_data

## Gráfica lineal

In [ ]:
#NORMALIZACION DE LOS DATOS
from sklearn.preprocessing import MinMaxScaler

# Crear un nuevo DataFrame con las columnas seleccionadas
data_to_normalize = combined_data[['N', 'P', 'NEU', 'close']]

scaler = MinMaxScaler()

# Aplicar la normalización a los datos seleccionados
normalized_data = pd.DataFrame(scaler.fit_transform(data_to_normalize), columns=data_to_normalize.columns)

# Unir los datos normalizados con la columna 'date'
normalized_data = pd.concat([combined_data['date'], normalized_data], axis=1)


In [ ]:
normalized_data

In [ ]:
import matplotlib.pyplot as plt

# Crear la figura y los ejes del gráfico
fig, ax = plt.subplots()

# Graficar la serie de sentimiento negativo
ax.plot(normalized_data['date'], normalized_data['N'], label='Sentimiento Negativo')

# Graficar la serie de sentimiento positivo
ax.plot(normalized_data['date'], normalized_data['P'], label='Sentimiento Positivo')

# Graficar la serie de sentimiento neutro
ax.plot(normalized_data['date'], normalized_data['NEU'], label='Sentimiento Neutro')

# Graficar la serie del valor medio de cierre
ax.plot(normalized_data['date'], normalized_data['close'], label='Valor Medio de Cierre')

# Etiquetas de los ejes
ax.set_xlabel('Fecha')
ax.set_ylabel('Valor')

# Título del gráfico
ax.set_title('Análisis de Sentimiento y Valor Medio de Cierre')

# Mostrar leyenda
ax.legend()

# Rotar las etiquetas del eje x para una mejor visualización
plt.xticks(rotation=45)

# Mostrar el gráfico
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Crear una figura y subgráficos individuales
fig, axs = plt.subplots(3, 1, figsize=(8, 12))

# Configurar los subgráficos
sentimientos = ['N', 'P', 'NEU']
colores = ['red', 'green', 'blue']

# Graficar cada sentimiento con el precio de cierre
for i, sentimiento in enumerate(sentimientos):
    ax = axs[i]
    
    # Graficar el sentimiento
    ax.plot(normalized_data['date'], normalized_data[sentimiento], color=colores[i], label=sentimiento.capitalize())
    
    # Graficar el precio de cierre
    ax.plot(normalized_data['date'], normalized_data['close'], color='black', label='Precio de Medio MES')
    
    # Etiquetas de los ejes
    ax.set_xlabel('Fecha')
    ax.set_ylabel('Valor')
    
    # Título del subgráfico
    ax.set_title(f'Análisis de Sentimiento vs. Precio de Cierre ({sentimiento.capitalize()})')
    
    # Mostrar leyenda
    ax.legend()

# Ajustar los espacios entre subgráficos
fig.tight_layout()

# Mostrar los gráficos
plt.show()


## Cálculo de estadísticas descriptivas:

In [ ]:
import numpy as np

# Calcular estadísticas descriptivas para cada sentimiento
positivo_stats = combined_data['P'].describe()
negativo_stats = combined_data['N'].describe()
neutro_stats = combined_data['NEU'].describe()

# Imprimir las estadísticas descriptivas
print("Estadísticas descriptivas del Sentimiento Positivo:")
print(positivo_stats)
print()

print("Estadísticas descriptivas del Sentimiento Negativo:")
print(negativo_stats)
print()

print("Estadísticas descriptivas del Sentimiento Neutro:")
print(neutro_stats)
print()


## Grafico de disperción


In [ ]:
import seaborn as sns

# Crear un DataFrame con las columnas de interés
data_for_correlation = normalized_data[[ 'P', 'N','NEU', 'close']]

# Calcular la matriz de correlación
correlation_matrix = data_for_correlation.corr()

# Crear un gráfico de dispersión de las variables de interés
sns.pairplot(data_for_correlation)
plt.show()

# Mostrar la matriz de correlación como un mapa de calor
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Matriz de correlación entre Sentimiento y Precio de Cierre')
plt.show()


In [ ]:
# Contar el número de comentarios por sentimiento
num_negativos = combined_data['N'].sum()
num_positivos = combined_data['P'].sum()
num_neutros = combined_data['NEU'].sum()

# Imprimir los resultados
print("Número de comentarios negativos:", num_negativos)
print("Número de comentarios positivos:", num_positivos)
print("Número de comentarios neutros:", num_neutros)


## Grafica respecto a la variacion del precio mes a mes

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Calcular la variación del precio de cierre entre mes y mes
combined_data['variacion_precio'] = combined_data['close'].pct_change()


In [ ]:
combined_data['variacion_precio'].describe()

In [ ]:

# Graficar la relación entre la cantidad de comentarios y la variación del precio
plt.figure(figsize=(10, 6))
#plt.scatter(combined_data['N'], combined_data['variacion_precio'], label='Negativo')
#plt.scatter(combined_data['P'], combined_data['variacion_precio'], label='Positivo')
plt.scatter(combined_data['NEU'], combined_data['variacion_precio'], label='Neutro')
plt.xlabel('Cantidad de Comentarios')
plt.ylabel('Variación del Precio de Cierre')
plt.title('Relación entre Cantidad de Comentarios y Variación del Precio')
plt.legend()
plt.show()

# Calcular la correlación entre la cantidad de comentarios y la variación del precio
correlation = combined_data[['N', 'P', 'NEU', 'variacion_precio']].corr()

# Imprimir la matriz de correlación
print("Matriz de correlación:")
print(correlation)


In [ ]:
import pandas as pd

combined_data['clasificacion'] = ''

# Clasificación mes a mes
for index, row in combined_data.iterrows():
    clasificacion = ''
    if row['N'] > row['P'] and row['N'] >= row['NEU']:
        clasificacion = 'Negativo'
    elif row['P'] > row['N'] and row['P'] >= row['NEU']:
        clasificacion = 'Positivo'
    else:
        clasificacion = 'Neutro'
    
    combined_data.at[index, 'clasificacion'] = clasificacion

print(combined_data[['date', 'N', 'P', 'NEU', 'clasificacion']])


In [ ]:
combined_data['clasificacion'] 
# Contar el número de comentarios por sentimiento
num_negativos = combined_data[combined_data['clasificacion'] == 'Negativo']
num_positivos = combined_data[combined_data['clasificacion'] == 'Positivo']
num_neutros = combined_data[combined_data['clasificacion'] == 'Neutro']

# Imprimir los resultados
print("Número de meses negativos:", len(num_negativos))
print("Número de meses positivos:", len(num_positivos))
print("Número de meses neutros:", len(num_neutros))

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Graficar la variación del precio en función de la clase mensual etiquetada
plt.figure(figsize=(10, 6))
plt.scatter(combined_data[combined_data['clasificacion'] == 'Negativo']['variacion_precio'], 
            combined_data[combined_data['clasificacion'] == 'Negativo']['clasificacion'], 
            color='red', label='Negativo')
plt.scatter(combined_data[combined_data['clasificacion'] == 'Positivo']['variacion_precio'], 
            combined_data[combined_data['clasificacion'] == 'Positivo']['clasificacion'], 
            color='green', label='Positivo')
plt.scatter(combined_data[combined_data['clasificacion'] == 'Neutro']['variacion_precio'], 
            combined_data[combined_data['clasificacion'] == 'Neutro']['clasificacion'], 
            color='blue', label='Neutro')
plt.xlabel('Variación del Precio')
plt.ylabel('Clase Mensual Etiquetada')
plt.title('Comparación de la Variación del Precio con la Clase Mensual Etiquetada')
plt.legend()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Calcular el promedio de la variación del precio por cada clase mensual
mean_variation = combined_data.groupby('clasificacion')['variacion_precio'].mean()

# Crear un gráfico de barras para comparar la variación del precio por clase mensual
plt.figure(figsize=(10, 6))
plt.bar(mean_variation.index, mean_variation)
plt.xlabel('Clase Mensual Etiquetada')
plt.ylabel('Promedio de Variación del Precio')
plt.title('Comparación de la Variación del Precio por Clase Mensual Etiquetada')
plt.ylim(min(mean_variation) - 0.05)
plt.show()


In [ ]:
import pandas as pd

# Filtrar los meses con clasificación positiva y variación de precio positiva
positive_months = combined_data[combined_data['clasificacion'] == 'Positivo']
positive_price_increase = positive_months[positive_months['variacion_precio']<0]

# Calcular el porcentaje de coincidencia
percentage = len(positive_price_increase) / len(positive_months) * 100

# Imprimir el resultado
print("Porcentaje de meses negativos con variación de precio Positivo: {:.2f}%".format(percentage))

# Crear una figura y ejes para el gráfico de dispersión
fig, ax = plt.subplots(figsize=(10, 6))

# Graficar los puntos para los meses positivos y aumento de precio
ax.scatter(positive_price_increase['date'], positive_price_increase['variacion_precio'], color='green')

# Etiquetas y título del gráfico
ax.set_xlabel('Fecha')
ax.set_ylabel('Variación del Precio')
ax.set_title('Meses Positivos con Aumento de Precio')

# Mostrar el gráfico
plt.show()


In [ ]:
positive_price_increase

In [ ]:
negative_months = combined_data[combined_data['clasificacion'] == 'Negativo']
positive_price_increase = positive_months[positive_months['variacion_precio'] < 0]
# Calcular el porcentaje de coincidencia


In [ ]:

len(positive_months)

In [ ]:
len(negative_months)

In [ ]:
# se pasa la fecha a formato datetime
d_filtradas['fecha'] = pd.to_datetime(tweets_data['fecha'])
bitcoin_data['fecha'] = pd.to_datetime(bitcoin_data['fecha'])

In [ ]:
import matplotlib.pyplot as plt

# Supongamos que los datos están en un DataFrame llamado "comentarios"
# con las columnas "fecha", "comentario" y "etiqueta"

# Agrupamos los comentarios por fecha y etiqueta
#comentarios_agrupados = d_filtradas.groupby(['fechas', 'label']).size().reset_index(name='cantidad')
# Agrupamos los datos por fecha y clase y calculamos la suma del valor
datos_agrupados  = d_filtradas.groupby(['fechas', 'label'])['content'].sum().reset_index()

# Utilizamos la función pivot de pandas para reorganizar los datos en un dataframe
datos_pivote  = datos_agrupados.pivot(index='fechas', columns='label', values='content')

# Creamos la gráfica de líneas
fig, ax = plt.subplots(figsize=(30,6))
ax.plot(datos_pivote.index, datos_pivote['P'], label='P')
ax.plot(datos_pivote.index, datos_pivote['NEU'], label='NEU')
ax.plot(datos_pivote.index, datos_pivote['N'], label='N')

# Gráficas superpuestas
#plt.plot(datos_agrupados['fechas'], datos_pivote, label='sentimiento', color='red')
#plt.plot(d_grafica_bit['fechas_mes'], valor_bitcoin, label='bitcoin')

ax.legend()
plt.xticks(rotation=90)


plt.show()

In [ ]:
datos_pivote.plot(kind='line')
plt.title('Análisis de Sentimiento')
plt.xlabel('Fecha')
plt.ylabel('Número de Comentarios por clase')

plt.xticks(rotation=90)


plt.show()


In [ ]:
d_grafica_bit=pd.read_csv('BTC_ALL_graph_coinmarketcap.csv', sep=';')
d_grafica_bit['timestamp'][0]

In [ ]:
fechas_b=[]
for date in d_grafica_bit['timestamp']:
    fechas_b.append((datetime.strptime(date, '%Y-%m-%dT%H:%M:%S.%fZ')).strftime('%Y-%m'))
d_grafica_bit['fechas']=fechas_b

In [ ]:
d_grafica_bit

In [ ]:
fechas_coinmarket=[]
formato = '%Y-%m-%dT%H:%M:%S.%fZ'
for i in d_grafica_bit['timestamp']:
    fechas_coinmarket.append((datetime.strptime(i, formato)).strftime('%Y-%m'))
d_grafica_bit['fecha_mes']=fechas_coinmarket
d_grafica_bit

In [ ]:
d_grafica_bit=d_grafica_bit.groupby(['fechas'])['close'].mean().reset_index()
d_grafica_bit

In [ ]:
d_grafica_bit[d_grafica_bit['fechas']=='2023-01']

In [ ]:
# se cogen solo las fechas desde 2013-01 a 2022-12
d_grafica_bit= d_grafica_bit.drop(d_grafica_bit.index[:19])
d_grafica_bit=d_grafica_bit.drop(d_grafica_bit.index[150:])

In [ ]:
d_grafica_bit

In [ ]:
datos_pivote

In [ ]:
df_merged=pd.merge(d_grafica_bit, datos_pivote,on='fechas', how='outer')

In [ ]:
df_merged

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Normalizar las columnas 'precio', 'comentarios_positivos', 'comentarios_negativos' y 'comentarios_neutros'
scaler = MinMaxScaler()
df_merged[['close', 'N', 'NEU', 'P']] = scaler.fit_transform(df_merged[['close', 'N', 'NEU', 'P']])

# Imprimir el resultado
df_merged

In [ ]:
# Crear una figura y un eje
fig, ax = plt.subplots(figsize=(40,10))

# Graficar el precio y los comentarios positivos
ax.plot(df_merged['fecha_mes'], df_merged['close'], label='Precio')
ax.plot(df_merged['fecha_mes'], df_merged['P'], label='P')
ax.plot(df_merged['fecha_mes'], df_merged['NEU'], label='NEU')
ax.plot(df_merged['fecha_mes'], df_merged['N'], label='N')
# Agregar leyenda y títulos de ejes
ax.legend()
ax.set_xlabel('Fecha')
ax.set_ylabel('Valor normalizado')
ax.set_title('Precio y comentarios positivos')
plt.xticks(rotation=90)
# Mostrar la gráfica
plt.show()
